In [29]:
#IMPORTANDO BIBLIOTECAS
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import sqlite3
import time
from tqdm import tqdm
import os
import random

In [30]:
def Random_user_agents(marketplace):

    #Criando a lista de user-agents
    User_agents_list = ['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36 OPR/85.0.4341.60',
                        'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.130 Safari/537.36',
                        'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.134 Safari/537.36',
                        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.130 Safari/537.36',
                        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.101 Safari/537.36',
                        'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.101 Safari/537.36',
                        'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.65 Safari/537.36',
                        'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.85 Safari/537.36',
                        'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36']

    #Escolhendo o user aleatório
    Choice_user_agent = random.choice(User_agents_list)

    headers = {'authority':'www.' + marketplace + '.com.br',
               'user-agent':Choice_user_agent}


    return headers

In [31]:
#GUARDANDO LISTAS
Links_Americanas = []
Urls_Americanas = []
Sellers_Americanas = []
Country_Americanas = []
Price_Americanas = []
SKU_Americanas = []
Title_Americanas = []
Installment_Americanas_quantidade = []
Installment_Americanas_valor_parcela = []
Installment_Americanas_valor_total = []
More_offers_americanas = []

In [32]:
def getting_n_creating_americanas_urls(brand):

    Database_path = r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1" + "\Data\\" + brand + "\\" + brand + ".db"

    table = brand + "_products"

    #Criando a Query
    query = "SELECT * FROM " + table

    #Entrando dentro do databse
    connection = sqlite3.connect(Database_path)

    #Criando o dataset em brando
    df = pd.read_sql_query(query, connection)

    #Passando todo o Dataframe para LowerCase
    df = df.apply(lambda x: x.astype(str).str.lower())

    #Arrumando espaços vazios
    # Arrumano os espaços vazios
    df['Product_Name'] = df['Product_Name'].str.replace(" ", "-")

    # Criando uma nova coluna no database com a formatação certa
    df['Urls'] = df['Brand'] + "-" + df['Product_Name']

    # Criando a nova coluna que são as urls de pesquisa
    df['Urls_search'] = "https://www.americanas.com.br/busca/" + df['Urls']

    return df

In [33]:
Df_Products_Links = getting_n_creating_americanas_urls("Wacom")
Df_Products_Links

,Brand,Product_Name,Product_Descrition,Urls,Urls_search
0,wacom,ctl472,ctl 472 - mesa digitalizadora one by wacom peq...,wacom-ctl472,https://www.americanas.com.br/busca/wacom-ctl472
1,wacom,ctl4100,ctl4100 - mesa digitalizadora wacom intuos sma...,wacom-ctl4100,https://www.americanas.com.br/busca/wacom-ctl4100
2,wacom,ctl4100wle0,ctl4100wle0 - mesa digitalizadora wacom intuos...,wacom-ctl4100wle0,https://www.americanas.com.br/busca/wacom-ctl4...
3,wacom,ctl4100wlk0,ctl4100wlk0 - mesa digitalizadora wacom intuos...,wacom-ctl4100wlk0,https://www.americanas.com.br/busca/wacom-ctl4...
4,wacom,ctl6100wlk0,ctl6100wlk0 - mesa digitalizadora wacom intuos...,wacom-ctl6100wlk0,https://www.americanas.com.br/busca/wacom-ctl6...
5,wacom,dtc133w0a,dtc133w0a - mesa digitalizadora wacom one crea...,wacom-dtc133w0a,https://www.americanas.com.br/busca/wacom-dtc1...
6,wacom,pth460,pth460 - mesa digitalizadora wacom intuos pro ...,wacom-pth460,https://www.americanas.com.br/busca/wacom-pth460
7,wacom,dtk1660k0a,dtk1660k0a - display interativo wacom cintiq 1...,wacom-dtk1660k0a,https://www.americanas.com.br/busca/wacom-dtk1...
8,wacom,pth660,pth660 - mesa digitalizadora wacom intuos pro ...,wacom-pth660,https://www.americanas.com.br/busca/wacom-pth660
9,wacom,ctl6100wle0,ctl6100wle0 - mesa digitalizadora wacom intuos...,wacom-ctl6100wle0,https://www.americanas.com.br/busca/wacom-ctl6...


In [34]:
def search_links(url):
    global Links_Americanas

    time.sleep(10)

    Headers_Choice = Random_user_agents("Americanas")

    response = requests.get(url, headers=Headers_Choice)
    
    if response.status_code != 200:
        Headers_Choice = Random_user_agents("Americanas")
        response = requests.get(url, headers=Headers_Choice)

    else:
        pass

    html = response.text

    bs = BeautifulSoup(html, 'html.parser')

    for link in bs.find_all("a", href=True):
        Links_Americanas.append("https://www.americanas.com.br" + link['href'])

    Links_Americanas = [s for s in Links_Americanas if 'produto' in s]
    Links_Americanas = [s for s in Links_Americanas if not 'theater' in s]

In [35]:
for url in tqdm(Df_Products_Links['Urls_search']):
    search_links(url)

100%|██████████| 10/10 [01:47<00:00, 10.76s/it]


In [36]:
def get_atributes(url):
    time.sleep(25)

    Headers_Choice = Random_user_agents("Americanas")

    response = requests.get(url, headers=Headers_Choice)

    if response.status_code != 200:
        
        Headers_Choice = Random_user_agents("Americanas")
        response = requests.get(url, headers=Headers_Choice)

    else:
        pass

    html = response.text

    soup = BeautifulSoup(html, 'html.parser')

    #Title
    try:
        Title_Americanas.append(soup.find(class_='product-title__Title-sc-1hlrxcw-0 jyetLr').text)
    except:
        Title_Americanas.append("Erro")

    #Preço
    try:
        Price_Americanas.append(soup.find(class_='styles__PriceText-sc-x06r9i-0 dUTOlD priceSales').text)
    except:
        Price_Americanas.append('Erro')

    #Installment
    try:
        Installment_Americanas_valor_parcela.append(soup.find(class_='payment-installment-text__Text-sc-12txe9z-0 bfFyfi').text)
    except:
        Installment_Americanas_valor_parcela.append("Erro")

    #Seller
    try:
        Sellers_Americanas.append(soup.find(class_='offers-box__Wrapper-sc-189v1x3-0 kegaFO').text)
    except:
        Sellers_Americanas.append("Erro")

    #More offers 
    try:
        More_offers_americanas.append(soup.find(class_='more-offers__Touchable-sc-15yqej3-2 hYfNEd').text)
    except:
        More_offers_americanas.append("Erro")

In [37]:
Links_Americanas

['https://www.americanas.com.br/produto/133710364?pfm_carac=wacom-ctl472&pfm_page=search&pfm_pos=grid&pfm_type=search_page&offerId=5f91be4f1b186381b569a65c&buyboxToken=smartbuybox-acom-v2-f66339ad-4c2f-413b-ae5e-c5a41235079b-2022-05-26 11:11:12-0300',
 'https://www.americanas.com.br/produto/3719526302?pfm_carac=wacom-ctl472&pfm_page=search&pfm_pos=grid&pfm_type=search_page&offerId=61183d2f09c351890dd3b363',
 'https://www.americanas.com.br/produto/4066305835?pfm_carac=wacom-ctl472&pfm_page=search&pfm_pos=grid&pfm_type=search_page&offerId=627ca27587c00289c26557b2',
 'https://www.americanas.com.br/produto/3839713911?pfm_carac=wacom-ctl472&pfm_page=search&pfm_pos=grid&pfm_type=search_page&offerId=6152465209c351890d0e2b5f',
 'https://www.americanas.com.br/produto/4798981505?pfm_carac=wacom-ctl472&pfm_page=search&pfm_pos=grid&pfm_type=search_page&offerId=6227d4bc87c00289c207750e',
 'https://www.americanas.com.br/produto/4958627240?pfm_carac=wacom-ctl472&pfm_page=search&pfm_pos=grid&pfm_type=

In [38]:
for url in tqdm(Links_Americanas):
    get_atributes(url)

100%|██████████| 113/113 [38:53<00:00, 20.65s/it]


In [59]:
def create_dataframe(url, sellers, price, installment, title):
    df_raw = pd.DataFrame()

    Hoje = pd.to_datetime('today', errors='ignore').date()

    df_raw['URL'] = url

    df_raw['DATE'] = Hoje

    df_raw['MARKETPLACE'] = 'Americanas'

    df_raw['SELLER'] = sellers
    df_raw['SELLER'] = df_raw['SELLER'].str.replace("Este produto é vendido e entregue por ","")
    df_raw['SELLER'] = df_raw['SELLER'].str.replace("Este produto é vendido por ","")
    df_raw['SELLER'] = df_raw['SELLER'].str.partition(".")[0]
    df_raw['SELLER'] = df_raw['SELLER'].str.partition(" e")[0]
    


    df_raw['PRICE'] = price
    df_raw['PRICE'] = df_raw['PRICE'].str.replace("R$ ", "", regex=False)
    df_raw['PRICE'] = df_raw['PRICE'].str.replace(".","")
    df_raw['PRICE'] = df_raw['PRICE'].str.replace(",",".")



    df_raw['Installment_full'] = installment
    df_raw['PARCEL'] = df_raw['Installment_full'].str.partition('x')[0].str.partition("até ")[2]

    df_raw['INSTALLMENT'] = df_raw['Installment_full'].str.partition("x")[2].str.partition("R$ ")[2]
    df_raw['INSTALLMENT'] = df_raw['INSTALLMENT'].str.replace(",",".")

    df_raw['PRODUCT'] = title

    df_raw['MORE'] = More_offers_americanas

    df_raw['ID'] = df_raw['URL'].str.partition("produto/")[2].str.partition('?')[0]

    df_raw = df_raw[['DATE', 'URL', 'MARKETPLACE', 'SELLER', 'PRICE', 'PARCEL', 'INSTALLMENT', 'ID', 'PRODUCT',  'MORE']]


    return df_raw

In [60]:
Dataset_Americanas = create_dataframe(Links_Americanas, Sellers_Americanas, Price_Americanas, Installment_Americanas_valor_parcela, Title_Americanas)

C:\Users\pedro\AppData\Local\Temp\ipykernel_43496\1739868269.py:22: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_raw['PRICE'] = df_raw['PRICE'].str.replace(".","")


In [61]:
Dataset_Americanas

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,ID,PRODUCT,MORE
0,2022-05-26,https://www.americanas.com.br/produto/13371036...,Americanas,FiberMega,194.39,4,55.22,133710364,Mesa Digitalizadora One By Wacom CTL472,26 outras ofertas deste mesmo produto
1,2022-05-26,https://www.americanas.com.br/produto/37195263...,Americanas,SHOPAR,298.90,5,59.78,3719526302,Mesa Digitalizadora Wacom One CTL472 Pequena B...,Erro
2,2022-05-26,https://www.americanas.com.br/produto/40663058...,Americanas,HD Store,328.88,6,54.81,4066305835,Mesa Digitalizadora Wacom Ctl472 One,Erro
3,2022-05-26,https://www.americanas.com.br/produto/38397139...,Americanas,Catarinas,644.00,8,80.50,3839713911,Mesa Digitalizadora CTL472 One Wacom,Erro
4,2022-05-26,https://www.americanas.com.br/produto/47989815...,Americanas,KADRI TECNOLOGIA,399.00,7,57.00,4798981505,Mesa digitalizadora CTL472 one by wacom redwood,Erro
...,...,...,...,...,...,...,...,...,...,...
108,2022-05-26,https://www.americanas.com.br/produto/48009631...,Americanas,Haylou,3080.14,8,427.79,4800963147,Mesa Digitalizadora Wacom Intuos Pro PTH660 co...,Erro
109,2022-05-26,https://www.americanas.com.br/produto/61058374...,Americanas,Tendinfo,6788.00,8,848.50,61058374,Mesa Digitalizadora Wacom Intuos Pro Média (pt...,Erro
110,2022-05-26,https://www.americanas.com.br/produto/48387443...,Americanas,MultiShop,3911.29,8,488.91,4838744302,Mesa Digitalizadora Wacom Intuos Pro PTH660 co...,Erro
111,2022-05-26,https://www.americanas.com.br/produto/23011597...,Americanas,MIRANDA,2037.91,8,254.73,2301159787,Mesa Digitalizadora Intuos Bluetooth Média - V...,Erro


In [62]:
Dataset_Americanas.to_excel(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\Americanas_Wacom.xlsx", index=False)